<img src="https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png" srcset="https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_130 130w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_260 260w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_390 390w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_520 520w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_650 650w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_780 780w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_910 910w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1040 1040w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1170 1170w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1290 1290w" sizes="100vw" width="1290">
<p style='margin-top: 1rem; margin-bottom: 1rem;'>Developed by Marco Frodl, Principal Consultant for Generative AI @ <a href='https://go.mfr.one/tt-en' _target='blank'>Thinktecture AG</a> -- More about me on my <a href='https://go.mfr.one/marcofrodl-en' _target='blank'>profile page</a></p>

We want to implement a RAG pipeline with two retrievers. The "movies" retriever holds information about James Bond movies, and the "episodes" retriever holds information about every episode of Star Trek season 1. You have to implement a MultiRetrievalQAChain with LangChain that takes the user's query, determines the best retriever, and attempts to answer the question.

Tasks and details:

* All parts where you should implement something are marked with "..." or a sentence starting with "Task: "
* Include tracing with Langfuse.
* Use Qdrant as a Vector DB in in-memory mode.
* Embedding model should be open source BGE-M3 - the needed implentation to use it via OVH is given as a ready to go class
* As LLM use open source Meta Llama-3.1-70B-Instruct from OVH via the OpenAI implementation of LangChain
* Start with some searches in the vector db
* Implement a simple qa chain as warmup
* For the MultiRetrievalQA chain the prompt should output the answer in English no matter what language the question was
* If the question doesn't fit to either movies or tv episodes the output should be "Nothing I can answer during MLCon 2024"
* Generate answers for these questions and try both languages:
  * Question 1: In welchem Film geht es um ein Gerät in der russischen Botschaft? Gib Titel, Hauptdarsteller und Erscheinungsjahr aus. - In which movie is there a device in the Russian embassy? Provide the title, main actors, and release year.
  * Question 2: In welcher Folge kommen Computer vor? - In which episode do computers appear?
  * Question 3: Wie backe ich einen Apfelkuchen? - How to bake an apple pie?

If you need help:
* Use the LangChain chat bot for anything about LangChain: https://chat.langchain.com/
* For general questions you can use free https://huggingface.co/chat/ with nvidia/Llama-3.1-Nemotron-70B-Instruct-HF LLM - you need a free HuggingFace account to login

### Set API keys via Colab Secrets

In [ ]:
# import Colab Secrets userdata module
from google.colab import userdata
import os

# set Langfuse API keys
os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get('LANGFUSE_PUBLIC_KEY')
os.environ["LANGFUSE_SECRET_KEY"] = ...

## Load libraries

In [ ]:
!pip -q install langchain==0.3.7 langchain-community==0.3.7 langchain-openai==0.2.9 langchain-qdrant==0.2.0 langfuse==2.53.9

In [ ]:
!pip show langchain
!pip show langchain-openai

## Prepare Langfuse

In [ ]:
from langfuse.callback import CallbackHandler
handler = ...

## Prepare Embedding Model

In [ ]:
# using embeddings from MistralAI oder OpenAI is super easy with LangChain - but you would need a paid api key
#from langchain_mistralai import MistralAIEmbeddings
#embeddings = MistralAIEmbeddings(mistral_api_key=userdata.get('MISTRAL_API_KEY'))

# we want to use free OVH AI endpoints. Therefore we use the LangChain base implementation for embeddings and create a small wrapper for OVH
# it's more complex than Mistral or OpenAI but free
from langchain_core.embeddings import Embeddings
import requests

class OVHEmbeddings(Embeddings):
    def __init__(self, api_url: str, access_token: str):
        self.api_url = api_url
        self.headers = {
            "Content-Type": "text/plain",
            "Authorization": f"Bearer {access_token}",
        }

    def embed_query(self, text: str) -> list:
        """Generate embeddings for a single query."""
        response = requests.post(self.api_url, data=text, headers=self.headers)
        if response.status_code == 200:
            return response.json()  # Ensure this returns a list of floats
        else:
            raise ValueError(f"Error from embedding API: {response.status_code}, {response.text}")

    def embed_documents(self, texts: list) -> list:
        """Generate embeddings for a list of documents."""
        # Check if the API supports batch processing
        # Example:
        # response = requests.post(self.api_url, json={"texts": texts}, headers=self.headers)
        # Ensure proper handling based on the API's documentation

        # Fallback to single calls if batching is not supported
        embeddings = []
        for text in texts:
            embeddings.append(self.embed_query(text))
        return embeddings

# Initialize the embedding model
api_url = "https://bge-m3.endpoints.kepler.ai.cloud.ovh.net/api/text2vec"
access_token = userdata.get('OVH_AI_ENDPOINTS_ACCESS_TOKEN')
embeddings = OVHEmbeddings(...)

## Prepare LLM

In [ ]:
# for this notebook we use the free OVH AI endpoints and get access to a bunch of different open source LLMs
# you must register for an account to get your own (free) access token
from langchain_openai import ChatOpenAI

TEMPERATURE = 0
MAX_TOKENS = 1500

# OVH AI Endpoints Overview: https://endpoints.ai.cloud.ovh.net/

# OVH Mistral-7B-0.2
#MODEL_NAME = "Mistral-7B-Instruct-v0.2"
#BASE_URL = "https://mistral-7b-instruct-v02.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1"

# OVH Mixtral 8x22B
#MODEL_NAME = "Mixtral-8x22B-Instruct-v0.1"
#BASE_URL = "https://mixtral-8x22b-instruct-v01.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1"

# OVH Llama3-70B
#MODEL_NAME = "Meta-Llama-3-70B-Instruct"
#BASE_URL = "https://llama-3-70b-instruct.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1"

# OVH Llama3-8B
#MODEL_NAME = "Meta-Llama-3-8B-Instruct"
#BASE_URL = "https://llama-3-8b-instruct.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1"

# OVH Llama3.1-70B
#MODEL_NAME = "Meta-Llama-3_1-70B-Instruct"
#BASE_URL = "https://llama-3-1-70b-instruct.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1"

# OVH Mamba Codestral 7B
#MODEL_NAME = "mamba-codestral-7B-v0.1"
#BASE_URL = "https://mamba-codestral-7b-v0-1.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1/"

# OVH CodeLlama-13B
#MODEL_NAME = "CodeLlama-13b-Instruct-hf"
#BASE_URL = "https://codellama-13b-instruct-hf.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1"

# Task: As LLM use open source Meta Llama-3.1-70B-Instruct from OVH via the OpenAI implementation of LangChain
llm = ChatOpenAI(model= ..., temperature= ..., max_tokens= ..., base_url= ..., api_key= ...)


# with these two lines the demo switches to MistralAI don't forget to import langchain-mistralai
#from langchain_mistralai import ChatMistralAI
#llm = ChatMistralAI(model="open-mistral-nemo", temperature=0, max_tokens=1500, api_key=userdata.get('MISTRAL_API_KEY'))

# with these two lines the demo switches to OpenAI
#from langchain_openai import ChatOpenAI
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=1500)

## Prepare demo data

In [ ]:
# data for movies
docs_movies = [
"Filmnummer: 1	Jahr: 1962	Hauptdarsteller: Sean Connery	Deutscher Titel: James Bond jagt Dr. No	Originaltitel: Dr. No Inhalt: Nachdem auf der jamaikanischen Insel Kreb Key der britische Agent Strangways ermordet wird und der routinemäßige Anruf seiner Sekretärin nach London ausbleibt, schlägt man beim Geheimdienst Ihrer Majestät Alarm. Man beauftragt den Agenten mit der Decknummer 007 nach Jamaika zu fliegen und herauszufinden weshalb keine Antwort gekommen ist. Nachdem die Mitteilung 007 im Casino erreicht hat, begibt sich dieser (zum ersten mal in der Filmgeschichte) ins Hauptquartier von Universal Export’s“ zu seinem Chef „M“. Dort erfährt Bond, nach dem üblichen Plausch mit Miss Moneypenny, seinen neuen Auftrag. Noch dazu bekommt er einen Vortrag über seine alte Beretta und erhält eine neue Waffe, eine Walther PPK, vom Waffenmeister Major Boothroyd (somit war ein neues Markenzeichen ála 007 geboren). Bereits am Flughafen wird James Bond beobachtet. Als ihm die Gegenseite einen Chauffeur zur Verfügung stellt, Bond diesen nach einer kurzen Auseinandersetzung ausschaltet fährt er weiter und lernt einige Freunde von Strangways kennen. Unter diesen Freunden wird Bond auch auf den verdächtigen Professor Dent aufmerksam, auf den er später noch treffen wird. Außerdem stößt Bond auf den Fischer Quarrel und lernt den amerikanischen CIA-Agenten Felix Leiter kennen, die später beide zu den besten Freunden von 007 gehören. Bond stattet Professor Dent einen weiteren Besuch ab, und ihm wird klar dass Strangeways herausgefunden hat, dass die Gesteinsproben von der Insel Crab Key radioaktiv verseucht sind. Dent ist jedoch absolut anderer Meinung und somit entlarvt Bond ihn als Lügner und tötet diesen letztendlich. Da man Strangway’s Tod mit dem seltsamen Asiaten Dr. No in Verbindung bringt, entwickelt 007 einen Plan und macht sich mit Quarrel auf den Weg zur privaten Insel von Dr. No: Crab Key. Dort treffen die beiden auf die wunderschöne Muscheltaucherin Honey Rider und werden einige Zeit später von Dr. No’s Männern erkannt und schaffen es immer wieder ihnen zu entkommen. Schließlich werden sie von einem „Feuer speienden Drachen“ erwischt und Quarrel kommt dabei ums Leben. James und Honey werden gefangen genommen und verbringen das Abendessen mit dem geheimnisvollen Dr. No. Dabei erfährt 007 einiges über seinen teuflischen Plan: Dr. No will die Raketenstarts der NASA von Crab Key aus manipulieren und somit Rache an die westliche Welt ausüben. Bond gelingt es jedoch den Chinesen zu stoppen, Honey zu retten und schlussendlich die Insel in die Luft zu sprengen",
"Filmnummer: 2	Jahr: 1963	Hauptdarsteller: Sean Connery	Deutscher Titel: Liebesgrüße aus Moskau Originaltitel: From Russia With Love Inhalt: Die Verbrecherorganisation PHANTOM will drei Fliegen mit einer Klappe schlagen: Sie wollen eine besonders wertvolle Dechiffriermaschine (Lector) aus dem russischen Konsulat in Istanbul bekommen, den britischen und russischen Geheimdienst gegeneinander ausspielen und den besten Agenten Ihrer Majestät, James Bond 007, töten und gleichzeitig an ihm wegen dessen Ermordung ihres besten Mitarbeiters Dr. No Rache nehmen. Deshalb benutzen sie Bond, auf den sie die hübsche Romanova abgesetzt haben, um an die Dechiffriermaschine zu gelangen. 007 riecht jedoch den Köder und setzt alles daran, PHANTOM zu überlisten und selbst mit der Lector zu fliehen. Bei seiner Flucht mit dem Orientexpress aus Istanbul Richtung Italien kommt es zu einem fulminanten Aufeinandertreffen zwischen Bond und dem Phantommitarbeiter Grant",
"Filmnummer: 3	Jahr: 1964	Hauptdarsteller: Sean Connery	Deutscher Titel: Goldfinger Originaltitel: Goldfinger Inhalt: Auric Goldfinger ist ein gewöhnlicher Juwelier. Oder doch nicht? James Bond soll herauszufinden wie bzw. ob Goldfinger neben seinen gut laufenden Geschäften auch Gold schmuggelt. 007 verfolgt Goldfinger über Miami, England bis in die Schweiz, wo Goldfinger Bond auf die Schliche kommt und ihn töten lassen will. Doch Bond kennt Goldfingers höllischen Plan: Er will die größte Bank der Welt, Fort Knox, atomar verseuchen",
"Filmnummer: 4	Jahr: 1965	Hauptdarsteller: Sean Connery	Deutscher Titel: Feuerball Originaltitel: Thunderball Inhalt: Der skrupellose Mitarbeiter „Largo“ der Verbrecherorganisation Spectre stiehlt ein Natoflugzeug mit 2 Atombomben an Board. Spectre’s Forderung: 300 Millionen britische Pfund oder es werden 2 Großstädte der Welt von ihnen vernichtet. Dem britischen Geheimdienst ist klar, für diesen Auftrag kann es nur einen geben: 007! Doch die Zeit ist knapp, das Ultimatum läuft in Kürze ab. Doch der frisch erhohlte James Bond folgt einer Spur und reist auf die Bahamas, wo er auf die wunderschöne Domino trifft",
"Filmnummer: 5	Jahr: 1967	Hauptdarsteller: Sean Connery	Deutscher Titel: Man lebt nur zweimal	Originaltitel: You Only Live Twice Inhalt: Als plötzlich eine amerikanische und eine sowjetische Weltraumkapsel von einem unbekannten Raumschiff entführt werden, beschuldigen sich beide Großmächte gegenseitig des Diebstahles. Ein Atomkrieg steht also kurz bevor. Der britische Geheimdienst MI6 glaubt jedoch zu wissen, wo das Raumschiff gelandet ist: in Japan. Der eigentlich für tot gehaltene James Bond wird daher nach Japan geschickt, und nimmt dort mit dem Chef des japanischen Geheimdienstes „Tiger Tanaka“ Kontakt auf. Beide stoßen bei ihren Untersuchungen schon bald auf einen alten Bekannten: Ernst Stavro Blofeld!",
"Filmnummer: 6  Jahr: 1969  Hauptdarsteller: George Lazenby  Deutscher Titel: Im Geheimdienst Ihrer Majestät Originaltitel: On Her Majesty’s Secret Service Inhalt: James Bond wird nach zweijähriger, erfolgloser Jagd auf Ernst Stavro Blofeld von seinem Auftrag, ihn zu finden, entbunden. Daraufhin kündigt Bond seinen Dienst und möchte den Secret Service verlassen. Miss Moneypenny können wir es jedoch verdanken, dass es nur 2 Wochen geworden sind und 007 auch in seinem Urlaub weiterhin Nachforschungen über Blofeld unternimmt. Seine Spur führt ihn in die Schweizer Alpen. Nebenbei verliebt sich Bond in die exzentrische, wunderschöne Tracy, was sein Leben nicht unbedingt leichter macht",
"Filmnummer: 7  Jahr: 1971  Hauptdarsteller: Sean Connery  Deutscher Titel: Diamantenfieber Originaltitel: Diamonds Are Forever Inhalt: Nach dem Tod seiner Frau sucht Bond nach seinem Erzfeind Ernst Stavro Blofeld um Rache zu nehmen. Was ihm auch gelingt… oder doch nicht? Wie es sich herausstellt, hat Blofeld dieses mal einige Doppelgänger. Jedenfalls wird James nach Amerika geschickt, um Diamantenschmugglern auf die Spur zu kommen. Und wiederum gibt es ein Treffen mit Blofeld. Schafft es 007, seinen Erzfeind dieses Mal endgültig außer Gefecht setzen?",
"Filmnummer: 8  Jahr: 1973  Hauptdarsteller: Roger Moore  Deutscher Titel: Leben und sterben lassen Originaltitel: Live And Let Die Inhalt: Als aus mysteriösen Gründen 3 Männer zur selben Zeit an verschiedenen Orten getötet werden, wird James Bond nach New York geschickt. Denn angeblich steckt hinter den ganzen Morden ein Drogenschmuggler namens „Mr. Big“. Dieser schwört auf eine Kartenleserin, Miss Solitär, die ihm stets seine Zukunft richtig voraussagt. Bis sie James Bond kennenlernt. Durch ihre Hilfe kommt 007 „Mr. Big“ auf die Spur und rettet die Welt vor einem „Drogen-Fiasko“.",
"Filmnummer: 9  Jahr: 2024  Hauptdarsteller: Roger Moore  Deutscher Titel: Der Mann mit dem goldenen Colt Originaltitel: The Man With The Golden Gun Inhalt: Als plötzlich eine goldene Kugel mit der Aufschrift 007 nach London zum britischen Geheimdienst geschickt wird, macht sich Bond auf die Suche nach dem Absender. Seine Reise führt ihn nach Hongkong und zum „bescheidenen“ Aufenthaltsort von Francisco Scaramanga, dem bestbezahltesten Killer der Welt, der jedes seiner Opfer mit einem Schuss ermordet. 007 soll der Nächste sein.",
"Filmnummer: 10  Jahr: 1977  Hauptdarsteller: Roger Moore  Deutscher Titel: Der Spion, der mich liebte Originaltitel: The Spy Who Loved Me Inhalt: Während Bond beim Skifahren in Österreich nur knapp einem Anschlag der Russen entgeht, wird ein U-Boot-Ortungsgerät von unbekannten Tätern gestohlen. Daraufhin beschließen England und Russland, ihre beiden Topagenten, James Bond und Anya Amasova auf den Fall anzusetzen. Ihre Untersuchungen führen sie zum Unterwasserlaboratorium von Reeder Stromberg.",
"Filmnummer: 11  Jahr: 1979  Hauptdarsteller: Roger Moore  Deutscher Titel: Moonraker – Streng geheim Originaltitel: Moonraker Inhalt: Als ein Moonraker-Shuttle vom Rücken einer Boing 707 gestohlen wird, wird Topagent 007 auf den Multimillionär und Moonraker-Besitzer Hugo Drax angesetzt. Bond erfährt, dass dieser die Weltbevölkerung ausrotten, und eine neue, eigene Rasse erschaffen will, welche im Weltall entstehen soll. Bond versucht alles, um dessen teuflischen Plan zu zerstören.",
"Filmnummer: 12  Jahr: 1981  Hauptdarsteller: Roger Moore  Deutscher Titel: In tödlicher Mission Originaltitel: For Your Eyes Only Inhalt: Als vor der griechischen Insel Korfu ein britisches Spionageschiff unter- und dabei ein Steuerungsgerät verloren- geht, wird Agent 007 auf diesen Auftrag angesetzt. Seine erste Fährte führt ihn zum Winterskigebiet Cortina d`Ampezzo. Der mit einer Frau (Melina) verbündete Bond, kommt dem Millionär Kristatos auf die Spur und verfolgt diesen bis auf ein Kloster auf einem riesen Felsen. Dort kommt es zum Showdown bei dem auch ein alter Bekannter wieder gesichtet wird: General Gogol!",
"Filmnummer: 13  Jahr: 1983  Hauptdarsteller: Roger Moore  Deutscher Titel: Octopussy Originaltitel: Octopussy Inhalt: Der in Berlin ermordete Agent 009 wird mit einem unbezahlbaren Fabergé-Ei in der Hand gefunden. Das Ei war jedoch eine Kopie, was aber bis dahin keiner wusste. Dem MI6 ist deshalb klar, dass jemand das Original um jeden Preis in seine Hände bekommen will. Glücklicherweise findet auch schon bald die nächste Auktion statt. Als ein Prinz Khan aus Indien das Ei um eine unbezahlbare Summer erwirbt, hängt sich Bond an seine Fersen an. Bonds Reise führt ihn zur geheimnisvollen Octopussy und schlussendlich auch wieder zurück nach Good-Old-Germany",
"Filmnummer: 14  Jahr: 1985  Hauptdarsteller: Roger Moore  Deutscher Titel: Im Angesicht des Todes Originaltitel: A View To A Kill Inhalt: Als es Bond gelingt, einen Mikrochip aus Sibirien mitzubringen, stellt sich heraus, dass dieser sogar eine Atomexplosion überstehen würde. Warum? Dieser Sache soll 007 auf den Grund gehen, und seine Spur führt ihn zum Psychopaten und Millionär Max Zorin. Gemeinsam mit seinem alten Kollegen und Pferdeliebhaber „Tibbet“ tarnt 007 als Adeliger und Chauffeur. Beide machen sich auf den Weg das Geheimnis zu lüften, warum Zorins Pferde bei jedem Rennen gewinnen. Es stellt sich heraus, dass diese alle einen Mikrochip implantiert haben. Bonds Reise führt ihn über Frankreich ins Silicon Valley und schlussendlich zur Golden Gate Bridge, wo es zum letzten Kampf zwischen Bond und Zorin kommt.",
"Filmnummer: 15  Jahr: 1987  Hauptdarsteller: Timothy Dalton  Deutscher Titel: Der Hauch des Todes Originaltitel: The Living Daylights Inhalt: Bond wird nach Bratislava geschickt um den russischen Agenten Koskov per Pipeline nach Österreich überlaufen zu lassen. Da der russische Agent nur knapp einem Anschlag entgangen ist, fährt Bond noch einmal nach Bratislava zurück und lernt dort die wunderschöne Kara kennen, die der Sniperschütze war, der Koskov hätte töten sollen. Als sich herausstellt, dass das mögliche Attentat auf Koskov nur gestellt war, fliehen beide wieder zurück nach Österreich und schließlich über Wien nach Tanger. Dort stellt sich heraus, dass einige Drogen- und Waffenschmuggler hinter der gesamten Affäre um Koskov stecken.",
"Filmnummer: 16  Jahr: 1989  Hauptdarsteller: Timothy Dalton  Deutscher Titel: Lizenz zum Töten Originaltitel: Licence To Kill Inhalt: Kurz vor der Hochzeit von Felix Leiter kommt die Meldung, das Drogenboss Sanchez per Flugzeug in der Nähe unterwegs sei. Diese Chance lassen sich die beiden, Bond und Leiter, nicht entgehen und fangen den Bösewicht. Dieser kommt aber einige Tage später wieder frei, tötet Leiters Frau und verletzt diesen selbst schwer! Was folgt ist ein beispielsloser Rachefeldzug vom verletzen und verärgerten James Bond… Nebenbei zerstört 007 noch Sanchez‘ riesiges Drogenlager und kann mit Hilfe einiger atemberaubenden LKW-Stunts Sanchez und dessen Handlanger schließlich zur Strecke bringen.",
"Filmnummer: 17  Jahr: 1995  Hauptdarsteller: Pierce Brosnan  Deutscher Titel: GoldenEye Originaltitel: GoldenEye Inhalt: Der für tot gehaltene Ex-Kollege von James Bond, 006 Alec Trevelyan, „steht von den Toten auf“ und versucht mit dem Super-Satelliten „Goldeneye“, der alle elektronische Geräte in einem Umkreis von einigen Kilometern ausschalten kann, den größten Diebstahl aller Zeiten zu unternehmen.",
]

#data for tv series episodes
# https://de.wikipedia.org/wiki/Raumschiff_Enterprise/Episodenliste
docs_episodes = [
"Folge: 1 Deutscher Titel: Das Letzte seiner Art Originaltitel: The Man Trap Erstausstrahlung US: 8. Sep. 1966 Erstausstrahlung DE: 28. Sep. 1987 Regie: Marc Daniels Inhalt: Die Crew der Enterprise trifft auf ein Wesen, das die Gestalt anderer annehmen kann und auf Salz angewiesen ist, um zu überleben.",
"Folge: 2 Deutscher Titel: Der Fall Charly Originaltitel: Charlie X Erstausstrahlung US: 15. Sep. 1966 Erstausstrahlung DE: 9. Feb. 1974 Regie: Lawrence Dobkin Inhalt: Ein junger Mann mit übernatürlichen Kräften wird zur Bedrohung für die Crew.",
"Folge: 3 Deutscher Titel: Die Spitze des Eisberges Originaltitel: Where No Man Has Gone Before Erstausstrahlung US: 22. Sep. 1966 Erstausstrahlung DE: 6. Okt. 1973 Regie: James Goldstone Inhalt: Nach einem Unfall entwickeln zwei Crewmitglieder gottähnliche Fähigkeiten.",
"Folge: 4 Deutscher Titel: Implosion in der Spirale Originaltitel: The Naked Time Erstausstrahlung US: 29. Sep. 1966 Erstausstrahlung DE: 5. Okt. 1987 Regie: Marc Daniels Inhalt: Ein Virus bringt die Crew dazu, ihre innersten Gefühle unkontrolliert auszuleben.",
"Folge: 5 Deutscher Titel: Kirk : 2 = ? Originaltitel: The Enemy Within Erstausstrahlung US: 6. Okt. 1966 Erstausstrahlung DE: 22. Juli 1972 Regie: Leo Penn Inhalt: Ein Transporterunfall teilt Captain Kirk in zwei gegensätzliche Persönlichkeiten.",
"Folge: 6 Deutscher Titel: Die Frauen des Mr. Mudd Originaltitel: Mudd’s Women Erstausstrahlung US: 13. Okt. 1966 Erstausstrahlung DE: 21. Sep. 1987 Regie: Harvey Hart Inhalt: Ein Schmuggler bringt drei verführerische Frauen an Bord, die ein Geheimnis verbergen.",
"Folge: 7 Deutscher Titel: Der alte Traum Originaltitel: What Are Little Girls Made Of? Erstausstrahlung US: 20. Okt. 1966 Erstausstrahlung DE: 19. Okt. 1987 Regie: James Goldstone Inhalt: Dr. Korby erschafft menschenähnliche Androiden, um die Menschheit zu verbessern.",
"Folge: 8 Deutscher Titel: Miri, ein Kleinling Originaltitel: Miri Erstausstrahlung US: 27. Okt. 1966 Erstausstrahlung DE: 2. Nov. 1987 Regie: Vincent McEveety Inhalt: Auf einem Planeten, der der Erde gleicht, hat eine Krankheit fast alle Erwachsenen ausgelöscht.",
"Folge: 9 Deutscher Titel: Der Zentralnervensystemmanipulator Originaltitel: Dagger of the Mind Erstausstrahlung US: 3. Nov. 1966 Erstausstrahlung DE: 26. Okt. 1987 Regie: Vincent McEveety Inhalt: Ein verrückter Wissenschaftler benutzt ein Gerät, um die Gedanken seiner Opfer zu kontrollieren.",
"Folge: 10 Deutscher Titel: Pokerspiele Originaltitel: The Corbomite Maneuver Erstausstrahlung US: 10. Nov. 1966 Erstausstrahlung DE: 7. Okt. 1972 Regie: Joseph Sargent Inhalt: Kirk blufft, um die Enterprise vor einer scheinbar übermächtigen Bedrohung zu retten.",
"Folge: 11 Deutscher Titel: Talos IV – Tabu – Teil 1 Originaltitel: The Menagerie – Part I Erstausstrahlung US: 17. Nov. 1966 Erstausstrahlung DE: 16. Nov. 1987 Regie: Marc Daniels Inhalt: Spock riskiert seine Karriere, um seinen ehemaligen Captain nach Talos IV zu bringen.",
"Folge: 12 Deutscher Titel: Talos IV – Tabu – Teil 2 Originaltitel: The Menagerie – Part II Erstausstrahlung US: 24. Nov. 1966 Erstausstrahlung DE: 23. Nov. 1987 Regie: Robert Butler Inhalt: Die Wahrheit hinter Spocks Aktionen und den Talosianern wird enthüllt.",
"Folge: 13 Deutscher Titel: Kodos, der Henker Originaltitel: The Conscience of the King Erstausstrahlung US: 8. Dez. 1966 Erstausstrahlung DE: 9. Nov. 1987 Regie: Gerd Oswald Inhalt: Kirk vermutet, dass ein Schauspieler ein berüchtigter Massenmörder ist.",
"Folge: 14 Deutscher Titel: Spock unter Verdacht Originaltitel: Balance of Terror Erstausstrahlung US: 15. Dez. 1966 Erstausstrahlung DE: 12. Okt. 1987 Regie: Vincent McEveety Inhalt: Die Enterprise tritt in einen taktischen Kampf gegen ein romulanisches Kriegsschiff.",
"Folge: 15 Deutscher Titel: Landeurlaub Originaltitel: Shore Leave Erstausstrahlung US: 29. Dez. 1966 Erstausstrahlung DE: 30. Nov. 1987 Regie: Robert Sparr Inhalt: Die Crew entdeckt einen Planeten, auf dem ihre Fantasien Realität werden.",
"Folge: 16 Deutscher Titel: Notlandung auf Galileo 7 Originaltitel: The Galileo Seven Erstausstrahlung US: 5. Jan. 1967 Erstausstrahlung DE: 14. Okt. 1972 Regie: Robert Gist Inhalt: Spock muss eine Rettungsmission auf einem gefährlichen Planeten leiten.",
"Folge: 17 Deutscher Titel: Tödliche Spiele auf Gothos Originaltitel: The Squire of Gothos Erstausstrahlung US: 12. Jan. 1967 Erstausstrahlung DE: 23. März 1974 Regie: Don McDougall Inhalt: Ein kindliches Wesen mit großen Kräften hält die Crew gefangen.",
"Folge: 18 Deutscher Titel: Ganz neue Dimensionen Originaltitel: Arena Erstausstrahlung US: 19. Jan. 1967 Erstausstrahlung DE: 7. Dez. 1987 Regie: Joseph Pevney Inhalt: Kirk kämpft gegen den Anführer einer reptilienartigen Rasse, um seine Crew zu retten.",
"Folge: 19 Deutscher Titel: Morgen ist Gestern Originaltitel: Tomorrow is Yesterday Erstausstrahlung US: 26. Jan. 1967 Erstausstrahlung DE: 27. Mai 1972 Regie: Michael O’Herlihy Inhalt: Ein Zeitunfall bringt die Enterprise ins 20. Jahrhundert.",
"Folge: 20 Deutscher Titel: Kirk unter Anklage Originaltitel: Court Martial Erstausstrahlung US: 2. Feb. 1967 Erstausstrahlung DE: 10. Juni 1972 Regie: Marc Daniels Inhalt: Kirk wird beschuldigt, ein Crewmitglied fahrlässig getötet zu haben.",
"Folge: 21 Deutscher Titel: Landru und die Ewigkeit Originaltitel: The Return of the Archons Erstausstrahlung US: 9. Feb. 1967 Erstausstrahlung DE: 21. Dez. 1987 Regie: Joseph Pevney Inhalt: Die Crew deckt die Kontrolle einer Gesellschaft durch einen Computergott auf.",
"Folge: 22 Deutscher Titel: Der schlafende Tiger Originaltitel: Space Seed Erstausstrahlung US: 16. Feb. 1967 Erstausstrahlung DE: 21. Okt. 1972 Regie: Marc Daniels Inhalt: Die Enterprise trifft auf Khan, einen genetisch verbesserten Tyrannen.",
"Folge: 23 Deutscher Titel: Krieg der Computer Originaltitel: A Taste of Armageddon Erstausstrahlung US: 23. Feb. 1967 Erstausstrahlung DE: 28. Dez. 1987 Regie: Joseph Pevney Inhalt: Zwei Planeten führen einen Krieg, der durch Computer simuliert wird.",
"Folge: 24 Deutscher Titel: Falsche Paradiese Originaltitel: This Side of Paradise Erstausstrahlung US: 2. März 1967 Erstausstrahlung DE: 4. Jan. 1988 Regie: Ralph Senensky Inhalt: Die Crew wird durch Sporen in einen Zustand völliger Zufriedenheit versetzt.",
"Folge: 25 Deutscher Titel: Horta rettet ihre Kinder Originaltitel: The Devil in the Dark Erstausstrahlung US: 9. März 1967 Erstausstrahlung DE: 11. Jan. 1988 Regie: Joseph Pevney Inhalt: Die Crew entdeckt ein Wesen, das zum Schutz seiner Nachkommen tötet.",
"Folge: 26 Deutscher Titel: Kampf um Organia Originaltitel: Errand of Mercy Erstausstrahlung US: 16. März 1967 Erstausstrahlung DE: 15. Dez. 1973 Regie: John Newland Inhalt: Die Enterprise vermittelt zwischen den Organianern und den Klingonen.",
"Folge: 27 Deutscher Titel: Auf Messers Schneide Originaltitel: The Alternative Factor Erstausstrahlung US: 23. März 1967 Erstausstrahlung DE: 14. Dez. 1987 Regie: Gerd Oswald Inhalt: Kirk begegnet einem Mann, der gegen sein Gegenstück aus einem Paralleluniversum kämpft.",
"Folge: 28 Deutscher Titel: Griff in die Geschichte Originaltitel: The City on the Edge of Forever Erstausstrahlung US: 6. Apr. 1967 Erstausstrahlung DE: 18. Jan. 1988 Regie: Joseph Pevney Inhalt: Kirk und Spock reisen durch die Zeit, um einen historischen Fehler zu verhindern.",
"Folge: 29 Deutscher Titel: Spock außer Kontrolle Originaltitel: Operation -- Annihilate! Erstausstrahlung US: 13. Apr. 1967 Erstausstrahlung DE: 25. Jan. 1988 Regie: Herschel Daugherty Inhalt: Die Crew bekämpft Parasiten, die die Kontrolle über Menschen übernehmen.",
]

## Create two vector db's (in-memory)

In [ ]:
from langchain_qdrant import QdrantVectorStore

# create in-memory vector db for Bond movies
store_movies = QdrantVectorStore.from_texts(
    docs_movies,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only just for the demo
    collection_name="movies",
    force_recreate=True,
)

# create in-memory vector db for Star Trek episodes
...

## Search in vectos db's

### Similarity search w/o score

In [ ]:
query = "Welcher Film beinhaltet eine Pistole mit Metalveredelung und von wann ist der Film?"
found_docs = store_movies.similarity_search(query)
print(found_docs[0].page_content)
print(found_docs[0].metadata)

In [ ]:
query = "In welcher Folge kommen Computer vor?" # In which episode do computers appear?
found_docs = store_episodes.similarity_search(query)
print(found_docs[0].page_content)
print(found_docs[0].metadata)

### Similarity search with score

In [ ]:
query = "Which movie is about a device in the Russian embassy? Who is the main actor and when was it released?"
found_docs = store_movies.similarity_search_with_score(query)
print(f"Ergebnisse: {len(found_docs)}")
document, score = found_docs[0]
print(document.page_content)
print(f"Score: {score}")
document, score = found_docs[1]
print(document.page_content)
print(f"Score: {score}")

In [ ]:
query = "In welcher Folge kommen Computer vor?"
found_docs = store_episodes.similarity_search_with_score(query)
print(f"Ergebnisse: {len(found_docs)}")
document, score = found_docs[0]
print(document.page_content)
print(f"Score: {score}")
document, score = found_docs[1]
print(document.page_content)
print(f"Score: {score}")

In [ ]:
query = "Ein junger Mann kommt nach Bel Air"
found_docs = store_movies.similarity_search_with_score(query,) #score_threshold=

if found_docs:
  document, score = found_docs[0]
  print(document.page_content)
  print(f"Score: {score}")
if not found_docs:
  print("Keine Ergebnisse zur Suche")

In [ ]:
query = "In welcher Folge kommen Computer vor?"
found_docs = store_episodes.similarity_search_with_score(query,k=3)
document, score = found_docs[0]
print(document.page_content)
print(document.metadata)
print(f"Score: {score}")
if len(found_docs) == 3:
  document, score = found_docs[1]
  print("----------------------")
  print(document.page_content)
  print(f"Score: {score}")
  document, score = found_docs[2]
  print("----------------------")
  print(document.page_content)
  print(f"Score: {score}")

## Simple Use-Case: Single Retriever

In [ ]:
# Create a RetrievalQA chain with the Qdrant retriever for movies
from langchain.chains import RetrievalQA

simple_qa_chain = RetrievalQA.from_chain_type(llm, retriever=store_movies.as_retriever(search_kwargs={"k":1,})) #"score_threshold"

# Get the answer from the QA chain
response = simple_qa_chain.invoke({"query":"Which movie is about a device in the Russian embassy? Who is the main actor and when was it released?"},{"callbacks":[handler]})

# Print the answer
print(response['result'])

In [ ]:
# Create a RetrievalQA chain with the Qdrant retriever for episodes - we can't mix

# Task: implement a simple qa chain here with the Qdrant retriever for episodes to answer this query: In welcher Folge kommen Computer vor? - In which episode do computers appear?

...

## Dynamically Selecting the best Retriever for Query with AI

In [ ]:
# Define Prompt for AI answers
# we can define multiple prompts and use one prompt per data source/retriever
from langchain import PromptTemplate

# Prompt template for German output
template_de = """Use the following pieces of context to answer the user's question as accurately, concisely, and specifically as possible.
Avoid including unrelated or speculative information, and base your answer strictly on the provided context.

If the answer cannot be explicitly found or directly inferred from the context, respond with: 'Diese Frage kann ich nicht beantworten.'

Write the answer in German.

{context}

Question: {question}

Answer:
"""

# Prompt template for English output
...

# Let's pick one template
prompt = PromptTemplate.from_template(...)

## List of Retrievers

In [ ]:
# https://github.com/langchain-ai/langchain/discussions/10828
# https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_retrieval_qa.MultiRetrievalQAChain.html
# https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/router/multi_retrieval_qa.py

retriever_infos_list = [
    {
        "name": "store_movies",
        "retriever": store_movies.as_retriever(search_kwargs={"k": 1}),
        "prompt": prompt,
        "description":"..."
    },
    {
    ...
    }]


### Default Chain

In [ ]:
# https://python.langchain.com/docs/modules/chains/how_to/custom_chain
from typing import Any, Dict, Optional, List
from langchain.callbacks.manager import (AsyncCallbackManagerForChainRun, CallbackManagerForChainRun)
from langchain.chains.base import Chain

class ThinktectureStringReturnChain(Chain):
    """
    A custom chain that returns just a string.
    """

    output_key: str = "result"
    default_response: str = "... <- this was your task ;-)"

    @property
    def input_keys(self) -> List[str]:
        """
        Will always return an empty list.
        """
        return []

    @property
    def output_keys(self) -> List[str]:
        """
        Will always return text key.
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that returns a fixed string
        response = self.default_response

        if run_manager:
            run_manager.on_text("Log something about this run - sync")

        return {self.output_key: response}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # Your custom chain logic goes here
        # This is just an example that returns a fixed string
        response = self.default_response

        if run_manager:
            await run_manager.on_text("Log something about this run - async")

        return {self.output_key: response}

    @property
    def _chain_type(self) -> str:
        return "string_return_chain"

thinktecture_default_chain = ThinktectureStringReturnChain()

## Define our MultiRetrievalQAChain

In [ ]:
# define MultiRetrievalQAChain
# https://python.langchain.com/api_reference/langchain/chains/langchain.chains.router.multi_retrieval_qa.MultiRetrievalQAChain.html#langchain.chains.router.multi_retrieval_qa.MultiRetrievalQAChain.from_retrievers

from langchain.chains.router.multi_retrieval_qa import MultiRetrievalQAChain
multi_retrieval_qa_chain = MultiRetrievalQAChain.from_retrievers(llm=llm, retriever_infos=..., default_chain=..., verbose=True)

## Test time

In [ ]:
response = multi_retrieval_qa_chain.invoke({"input": ...},{"callbacks":[...]})
print(response['result'])